In [ ]:
using CairoMakie

import LilGuys as lguys
using Arya
using LilGuys

In [ ]:
function sigma_axis(gp; limits=(-1, 2, -2, 2.5), kwargs...) 
	ax = Axis(gp, 
		xlabel=log_r_label,
		ylabel = log_sigma_label,
        limits=limits
		;kwargs...
	)

	return  ax
end

In [ ]:
function load_profiles(modelname, starsname; extension="stellar_profiles.hdf5")
    path = joinpath(modelsdir, modelname, "stars", starsname, extension)

    parampath = joinpath(modelsdir, modelname, "../stars", starsname, "profile.toml")
        
    if isfile(parampath)
        expected = LilGuys.load_profile(parampath)
    else
        @warn "$parampath not found"
        expected = nothing
    end

    profs = LilGuys.read_structs_from_hdf5(path, LilGuys.StellarProfile)
    idxs = parse.(Int, first.(profs))
    profs = last.(profs)

    s = sortperm(idxs)
    return [idxs[i] => profs[i] for i in s ], expected
end

In [ ]:
function compare_profiles(profiles, expected=nothing; 
        sequential=false, legend=true, limits=(-2.5, 1.0, -6, 3), 
        skip=1,
        kwargs...
    )
    
    fig = Figure()
	ax = sigma_axis(fig[1, 1]; limits=limits, kwargs...)

    plot_kwargs = Dict{Symbol, Any}()

    if sequential
        if profiles[1].second.time !== NaN
            has_time = true
            plot_kwargs[:colorrange] = (profiles[1].second.time * T2GYR, profiles[end].second.time * T2GYR)
        else
            
            plot_kwargs[:colorrange] = (1, length(profiles))
            has_time = false
        end
        
    end

    local p
    
    for i in eachindex(profiles)[1:skip:end]
        label, prof = profiles[i]
        x = prof.log_r
        y = prof.log_Sigma
        if sequential
            if has_time
                plot_kwargs[:color] = prof.time * T2GYR
            else
                plot_kwargs[:color] = i
            end
            
        end

    	p = lines!(x, y, label=string(label); plot_kwargs...)
    end

    if expected !== nothing
    	log_r_pred = LinRange(-2, 2, 1000)
    	ρ_s_pred = lguys.calc_Σ.(expected, 10 .^ log_r_pred)

    	lines!(log_r_pred, log10.(ρ_s_pred), label="expected", color="black", linestyle=:dot)
    end

    if legend
    	axislegend(ax, position=:lb)
    end 
    if sequential
        if has_time 
            label = "time / Gyr"
        else
            label = "snapshot"
        end
        
        Colorbar(fig[1,2], colorrange=plot_kwargs[:colorrange], label=label)
    end
    
	fig
end

In [ ]:
function plot_v_t!(profiles; kwargs...)
	sigmas = [p.second.sigma_vx for p in profiles]

    t = [p.second.time for p in profiles] 
	scatterlines!(t * T2GYR, sigmas * V2KMS; kwargs...)
end

In [ ]:
function plot_v_t(profiles; kwargs...)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}";
        kwargs...
	)

    plot_v_t!(profiles)

    fig
end
    

In [ ]:
function compare_v_t(profilesi)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}",
	)


    for i in eachindex(profilesi)
        label, profiles = profilesi[i]

        kwargs = Dict{Symbol, Any}()
        kwargs[:label] = label
  
        plot_v_t!(profiles; kwargs...)
    end

    axislegend()
	fig
end

In [ ]:
function plot_r_percens(profiles; kwargs...)
	fig = Figure()
	ax = Axis(fig[1,1], xlabel="time / Gyr", ylabel="log r / kpc containing stellar mass"; kwargs...)

	q = profiles[1].second.quantiles
	t = [p.second.time for p in profiles] * T2GYR
    
	for i in reverse(eachindex(q))
		Mq = [log10.(p.second.r_quantile[i]) for p in profiles]
        
		lines!(t, Mq, color=q[i], colorrange=extrema(q), label="$(q[i])")
	end

	Legend(fig[1, 2], ax, "quantile")
	fig
end
    

In [ ]:
"""
    all_plots(profiles[, expected])

Given the profiles object, this function displayes the plots
for the profiles and r percentiles for each timestep in the profile

"""
function all_plots(profiles, expected=nothing; prof_limits=(-2, 1, -15, 3), title="")
   display(compare_profiles(profiles, expected, 
            sequential=true, legend=false, title=title, limits=prof_limits,
        )
    )
    
    display(plot_r_percens(profiles, title=title))

    display(plot_v_t(profiles, title=title))

end

In [ ]:
prof_expected = lguys.StellarProfile("/astro/dboyea/dwarfs/observations/sculptor/processed/fiducial_sample_profile.toml");

In [ ]:
log_r_label = "log r / kpc"

In [ ]:
log_sigma_label = L"$\log \Sigma$ / N\,kpc$^{-2}$"

In [ ]:
modelsdir = ENV["DWARFS_ROOT"] * "/analysis/"

## V=32, r=5.9 (the initial model)

In [ ]:
models = []

In [ ]:
readdir(joinpath(modelsdir, "sculptor/1e6_V31_r3.2/orbit1/stars/exp2d_rs0.10"))

In [ ]:
profs, expected = load_profiles("sculptor/1e6_V31_r3.2/", "exp2d_rs0.10");


In [ ]:
compare_profiles(profs, expected, sequential=true, legend=false)

In [ ]:
profs, expected = load_profiles("sculptor/1e6_V31_r3.2/", "plummer_rs0.15");


In [ ]:
compare_profiles(profs, expected, sequential=true, legend=false)

In [ ]:
profs, expected = load_profiles("sculptor/1e6_V31_r3.2/orbit1", "exp2d_rs0.10");


Fiducial ish model

In [ ]:
profs, expected = load_profiles("sculptor/1e6_V31_r3.2/orbit_mean", "exp2d_rs0.10");


In [ ]:
compare_profiles(profs, expected, sequential=true, legend=false)

In [ ]:
profs, expected = load_profiles("sculptor/1e6_V31_r3.2/vasiliev+21_heavylmc_smallperilmc", "exp2d_rs0.13");


In [ ]:
compare_profiles(profs, expected, sequential=true, legend=false)

In [ ]:
profs, expected = load_profiles("sculptor/1e6_V31_r3.2/vasiliev+21_heavylmc", "exp2d_rs0.13");


In [ ]:
compare_profiles(profs, expected, sequential=true, legend=false)

In [ ]:
profs, expected = load_profiles("sculptor/1e6_V31_r3.2/vasiliev24_L3M11_smallperi/", "exp2d_rs0.13");


In [ ]:
compare_profiles(profs, expected, sequential=true, legend=false, skip=10, limits=(-2, 1, -7, 2))